In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_5"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold_current  Temp_Very_Cold_prev  \
7223  317.475875                       0                  0.0   
7224  383.628682                       0                  0.0   
7225  305.523072                       0                  0.0   
7226  308.408168                       0                  0.0   
7227  355.518770                       0                  0.0   
...          ...                     ...                  ...   
9037  297.867853                       0                  0.0   
9038  290.024007                       0                  0.0   
9039  335.823557                       0                  0.0   
9040  324.965348                       0                  0.0   
9041  261.341313                       0                  0.0   

      Temp_Very_Cold_next  Temp_Cold_current  Temp_Cold_prev  Temp_Cold_next  \
7223                  0.0                  0             0.0             0.0   
7224                  0.0                  0             0.

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Zeilen- und Spaltenanzeige in Pandas erweitern
pd.set_option("display.max_rows", None)  # Zeigt alle Zeilen
pd.set_option("display.max_columns", None)  # Zeigt alle Spalten

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                            1.000000
Silvester_current                 0.752698
zwischen_den_jahren_prev          0.373205
zwischen_den_jahren_current       0.365729
is_holiday_prev                   0.213621
Sommer_prev                       0.210736
Sommer_current                    0.208068
Sommer_next                       0.207425
Monat_8_next                      0.205846
Monat_8_current                   0.201454
is_holiday_current                0.200365
Monat_8_prev                      0.199771
is_holiday_next                   0.198810
Montag_next                       0.147877
Samstag_prev                      0.146234
Samstag_current                   0.145132
Freitag_prev                      0.141420
Sonntag_current                   0.141359
Sonntag_next                      0.116724
Temp_Warm_prev                    0.109710
Temp_Warm_current                 0.102382
Number_of_ships_current           0.101397
Number_of_ships_scaled_current    0.101397
Temp_Warm_n

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)



VPI_next                          0.171300
VPI_current                       0.153523
VPI_prev                          0.144343
Temp_Cold_prev                    0.087376
Temp_Cold_current                 0.074531
Temp_Cold_next                    0.068860
Sommer_prev                       0.068756
Sommer_current                    0.065597
Sommer_next                       0.061805
Werktag_current                   0.059766
Monat_8_prev                      0.050999
Monat_8_next                      0.050971
Monat_8_current                   0.050012
Winter_current                    0.049842
Winter_next                       0.048593
Winter_prev                       0.047252
Weihnachtsmarkt_prev              0.046687
Weihnachtsmarkt_current           0.045959
Monat_12_current                  0.044313
Montag_next                       0.042057
Weihnachtsmarkt_next              0.039668
Monat_12_next                     0.039428
Sonntag_current                   0.039256
Monat_12_pr

In [4]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Umsatz', 'Temp_Cold_current', 'Temp_Cold_prev', 'Temp_Cold_next', 'Cloud_Cloudy_current', 'Cloud_Cloudy_prev', 'Cloud_Cloudy_next', 'Wind_Light_current', 'Wind_Light_prev', 'Wind_Light_next', 'Wind_Moderate_current', 'Wind_Moderate_prev', 'Wind_Moderate_next', 'Wind_Strong_current', 'Wind_Strong_prev', 'Wind_Strong_next', 'KielerWoche_current', 'KielerWoche_prev', 'KielerWoche_next', 'Montag_current', 'Montag_prev', 'Montag_next', 'Dienstag_current', 'Dienstag_prev', 'Dienstag_next', 'Mittwoch_current', 'Mittwoch_prev', 'Mittwoch_next', 'Donnerstag_current', 'Donnerstag_prev', 'Donnerstag_next', 'Freitag_current', 'Freitag_prev', 'Freitag_next', 'Samstag_current', 'Samstag_prev', 'Samstag_next', 'Sonntag_current', 'Sonntag_prev', 'Sonntag_next', 'VPI_current', 'VPI_prev', 'VPI_next', 'Number_of_ships_current', 'Number_of_ships_prev', 'Number_of_ships_next', 'Ship_current', 'Ship_prev', 'Ship_next', 'is_holiday_current', 'is_holiday_prev', 'is_holiday_next'

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                            Feature           VIF
0            Temp_Very_Cold_current           inf
1               Temp_Very_Cold_prev           inf
2               Temp_Very_Cold_next           inf
3                 Temp_Cold_current           inf
4                    Temp_Cold_prev  3.790949e+07
5                    Temp_Cold_next           inf
6                 Temp_Mild_current           inf
7                    Temp_Mild_prev           inf
8                    Temp_Mild_next           inf
9                 Temp_Warm_current           inf
10                   Temp_Warm_prev           inf
11                   Temp_Warm_next           inf
12                 Temp_Hot_current           inf
13                    Temp_Hot_prev           inf
14                    Temp_Hot_next           inf
15              Cloud_Clear_current           inf
16                 Cloud_Clear_prev           inf
17                 Cloud_Clear_next           inf
18      Cloud_Partly_Cloudy_current           inf


In [6]:
# suche miteinander korrelierrende features

In [7]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Monat_8_prev', 'Monat_8_current'), ('Sommer_current', 'Temp_Warm_current'), ('Monat_12_current', 'Weihnachtsmarkt_current'), ('Number_of_ships_scaled_current', 'Ship_current'), ('Monat_9_next', 'Monat_9_current'), ('Monat_6_next', 'Monat_6_prev'), ('Temp_Cold_next', 'Temp_Cold_current'), ('Monat_3_current', 'Frühling_next'), ('Monat_6_next', 'Monat_6_current'), ('Samstag_current', 'Freitag_prev'), ('Temp_Warm_next', 'Temp_Warm_current'), ('Monat_5_next', 'Monat_5_current'), ('Monat_3_current', 'Frühling_current'), ('Winter_next', 'Winter_prev'), ('zwischen_den_jahren_prev', 'Silvester_prev'), ('Freitag_next', 'Mittwoch_prev'), ('Monat_8_current', 'Sommer_next'), ('Number_of_ships_scaled_next', 'Number_of_ships_next'), ('wetter_sehr_schön_current', 'Temp_Hot_next'), ('Monat_1_prev', 'Winter_prev'), ('Winter_current', 'Temp_Mild_next'), ('Monat_2_prev', 'Winter_next'), ('Monat_6_current', 'KielerWoche_prev'), ('Winter_current', 'Temp_Mild_current'), ('